In [55]:
import pandas as pd

In [4]:
ls -lh ../wordNet/

total 76M
-rw-r--r-- 1 fogside fogside 4,5M май 13  2017 composed_of.xml
-rw-r--r-- 1 fogside fogside  21M май 13  2017 derived_from.xml
-rw-r--r-- 1 fogside fogside 2,7M май 13  2017 senses.A.xml
-rw-r--r-- 1 fogside fogside  13M май 13  2017 senses.N.xml
-rw-r--r-- 1 fogside fogside 5,5M май 13  2017 senses.V.xml
-rw-r--r-- 1 fogside fogside 3,8M май 13  2017 synset_relations.A.xml
-rw-r--r-- 1 fogside fogside 8,4M май 13  2017 synset_relations.N.xml
-rw-r--r-- 1 fogside fogside 2,1M май 13  2017 synset_relations.V.xml
-rw-r--r-- 1 fogside fogside 3,1M май 13  2017 synsets.A.xml
-rw-r--r-- 1 fogside fogside 8,9M май 13  2017 synsets.N.xml
-rw-r--r-- 1 fogside fogside 3,2M май 13  2017 synsets.V.xml


In [161]:
from nltk.corpus.reader import wordnet

In [166]:
nltk.data.find('corpora/wordnet')

FileSystemPathPointer('/home/fogside/nltk_data/corpora/wordnet')

In [165]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/fogside/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [163]:
import nltk

In [8]:
import json

In [92]:
with open("../wordNet/synset_relations.N.json") as fn:
    relN = json.load(fn)

In [17]:
with open("../wordNet/synsets.N.json") as fn:
    senN = json.load(fn)

In [50]:
with open("../wordNet/senses.N.json") as fn:
    sensesN = json.load(fn)

In [54]:
sensesN['senses']['sense'][0]

{'#tail': '\n  ',
 '@id': '115643',
 '@lemma': 'МЕДИЦИНСКИЙ КОДИРОВАНИЕ',
 '@main_word': 'КОДИРОВАНИЕ',
 '@meaning': '1',
 '@name': 'МЕДИЦИНСКОЕ КОДИРОВАНИЕ',
 '@poses': 'Adj N',
 '@synset_id': 'N12658',
 '@synt_type': 'NG'}

In [98]:
relN['relations']['relation'][0]

{'#tail': '\n  ',
 '@child_id': 'N37195',
 '@name': 'hypernym',
 '@parent_id': 'N12658'}

In [100]:
len(relN['relations']['relation'])

130926

In [21]:
senN['synsets'].keys()

dict_keys(['synset', '#text'])

In [22]:
len(senN['synsets']['synset'])

29326

In [120]:
from tqdm import tqdm_notebook

In [138]:
relations = {}
for p in tqdm_notebook(relN['relations']['relation'][:30]):
    if p['@parent_id'] not in relations:
        main = set(lol[lol.main_id_x == p['@parent_id']].\
                   loc[:,['definition','root_name','text','lemma','main_word','name']].values.flatten())
        relations[p['@parent_id']]={}
        relations[p['@parent_id']]['main_syns'] = main
        relations[p['@parent_id']]['main_word'] = set(lol[lol.main_id_x == p['@parent_id']]['main_word'].
                                                      values.flatten()),
        relations[p['@parent_id']]['child_syns'] = []
        
    childs = set(lol[lol.main_id_x == p['@child_id']].\
               loc[:,['definition','root_name','text','lemma','main_word','name']].values.flatten())
    relations[p['@parent_id']]['child_syns'].append(childs)

HBox(children=(IntProgress(value=0, max=30), HTML(value='')))

In [150]:
relations['N12665']

{'child_syns': [{'обращаться к нескольким лицам с вопросом или несколькими вопросами с целью получения определённых сведений',
   'опрашивание',
   'опрос',
   'опрос, опрашивание'},
  set()],
 'main_syns': {'анкетирование', 'опрос при помощи анкет'},
 'main_word': ({'анкетирование'},)}

In [140]:
relations.keys()

dict_keys(['N12658', 'N12659', 'N12660', 'N12661', 'N12662', 'N12663', 'N12664', 'N12665', 'N12666', 'N12667'])

In [83]:
senses = []

for sen in sensesN['senses']['sense']:
    senses.append({'id':sen['@id'],
                   'meaning':sen['@meaning'].lower(),
                   'main_id':sen['@synset_id'].lower(),
                   'main_word':sen['@main_word'].lower() if len(sen['@main_word'])>0 else sen['@name'].lower(),
                   'lemma':sen['@lemma'].lower(),
                   'name':sen['@name'].lower()
                  })
    
sense_df = pd.DataFrame(senses)
sense_df

,id,lemma,main_id,main_word,meaning,name
0,115643,медицинский кодирование,n12658,кодирование,1,медицинское кодирование
1,115640,кодирование,n12658,кодирование,3,кодирование
2,115641,кодирование от зависимость,n12658,кодирование,1,кодирование от зависимости
3,115642,кодирование зависимость,n12658,кодирование,1,кодирование зависимости
4,117307,подготовка,n12659,подготовка,4,подготовка
5,117313,подготавливание,n12659,подготавливание,4,подготавливание
6,29834,полог,n12660,полог,1,полог
7,96991,сворачивание,n12661,сворачивание,4,сворачивание
8,96989,свертывание,n12661,свертывание,4,свертывание
9,50632,облавный охота,n12662,охота,1,облавная охота


In [24]:
senN['synsets']['synset'][1]

{'#tail': '\n  ',
 '#text': '\n    ',
 '@definition': 'готовить, настраивать соответствующим образом для восприятия чего-либо (какого-либо известия, сообщения и т. п.)',
 '@id': 'N12659',
 '@part_of_speech': 'N',
 '@ruthes_name': 'ПОДГОТОВИТЬ К ИЗВЕСТИЮ',
 'sense': [{'#tail': '\n    ', '#text': 'ПОДГОТОВКА', '@id': '117307'},
  {'#tail': '\n  ', '#text': 'ПОДГОТАВЛИВАНИЕ', '@id': '117313'}]}

In [74]:
from copy import deepcopy

In [27]:
sen=senN

In [75]:
synsets = []

for synset in sen['synsets']['synset']:
    tmp = {}
    tmp['main_id'] = synset['@id']
    tmp['definition'] = synset['@definition']
    tmp['root_name'] = synset['@ruthes_name'].lower()
    if (type(synset['sense']) is list):
        for syn in synset['sense']:
            tmp['text'] = syn['#text'].lower()
            tmp['id'] = syn['@id']
            synsets.append(deepcopy(tmp))
    elif type(synset['sense']) is dict:
        tmp['text'] = synset['sense']['#text'].lower()
        tmp['id'] = synset['sense']['@id']
        synsets.append(deepcopy(tmp))

In [86]:
sense_df.head()

,id,lemma,main_id,main_word,meaning,name
0,115643,медицинский кодирование,n12658,кодирование,1,медицинское кодирование
1,115640,кодирование,n12658,кодирование,3,кодирование
2,115641,кодирование от зависимость,n12658,кодирование,1,кодирование от зависимости
3,115642,кодирование зависимость,n12658,кодирование,1,кодирование зависимости
4,117307,подготовка,n12659,подготовка,4,подготовка


In [85]:
syns_df = pd.DataFrame(synsets)
syns_df.head()

,definition,id,main_id,root_name,text
0,,115643,N12658,кодирование от зависимости,медицинский кодирование
1,,115640,N12658,кодирование от зависимости,кодирование
2,,115641,N12658,кодирование от зависимости,кодирование от зависимость
3,,115642,N12658,кодирование от зависимости,кодирование зависимость
4,"готовить, настраивать соответствующим образом ...",117307,N12659,подготовить к известию,подготовка


In [87]:
lol = pd.merge(syns_df, sense_df, how='left', left_on='id', right_on='id')
lol.head()

,definition,id,main_id_x,root_name,text,lemma,main_id_y,main_word,meaning,name
0,,115643,N12658,кодирование от зависимости,медицинский кодирование,медицинский кодирование,n12658,кодирование,1,медицинское кодирование
1,,115640,N12658,кодирование от зависимости,кодирование,кодирование,n12658,кодирование,3,кодирование
2,,115641,N12658,кодирование от зависимости,кодирование от зависимость,кодирование от зависимость,n12658,кодирование,1,кодирование от зависимости
3,,115642,N12658,кодирование от зависимости,кодирование зависимость,кодирование зависимость,n12658,кодирование,1,кодирование зависимости
4,"готовить, настраивать соответствующим образом ...",117307,N12659,подготовить к известию,подготовка,подготовка,n12659,подготовка,4,подготовка


In [89]:
lol

,definition,id,main_id_x,root_name,text,lemma,main_id_y,main_word,meaning,name
0,,115643,N12658,кодирование от зависимости,медицинский кодирование,медицинский кодирование,n12658,кодирование,1,медицинское кодирование
1,,115640,N12658,кодирование от зависимости,кодирование,кодирование,n12658,кодирование,3,кодирование
2,,115641,N12658,кодирование от зависимости,кодирование от зависимость,кодирование от зависимость,n12658,кодирование,1,кодирование от зависимости
3,,115642,N12658,кодирование от зависимости,кодирование зависимость,кодирование зависимость,n12658,кодирование,1,кодирование зависимости
4,"готовить, настраивать соответствующим образом ...",117307,N12659,подготовить к известию,подготовка,подготовка,n12659,подготовка,4,подготовка
5,"готовить, настраивать соответствующим образом ...",117313,N12659,подготовить к известию,подготавливание,подготавливание,n12659,подготавливание,4,подготавливание
6,,29834,N12660,полог,полог,полог,n12660,полог,1,полог
7,,96991,N12661,свернуть (сломать),сворачивание,сворачивание,n12661,сворачивание,4,сворачивание
8,,96989,N12661,свернуть (сломать),свертывание,свертывание,n12661,свертывание,4,свертывание
9,"охота, при которой загонщики окружают то место...",50632,N12662,облава на зверя,облавный охота,облавный охота,n12662,охота,1,облавная охота


In [90]:
lol.sort_values(by='main_word')

,definition,id,main_id_x,root_name,text,lemma,main_id_y,main_word,meaning,name
3738,,1,N14050,австралийский доллар,aud,aud,n14050,aud,1,aud
9137,,3,N15988,контактный центр,call-центр,call-центр,n15988,call-центр,1,call-центр
30103,денежная единица в Европейском союзе,9,N24098,евро,eur,eur,n24098,eur,1,eur
59056,денежная единица Великобритании и ряда других ...,13,N35004,фунт стерлингов,gpb,gpb,n35004,gpb,1,gpb
10313,,39,N16407,интернет,internet,internet,n16407,internet,1,internet
10904,,53,N16605,информационные технологии,it-технология,it-технология,n16605,it-технологии,1,it-технологии
45632,денежная единица Японии,59,N29983,японская иена,jpy,jpy,n29983,jpy,1,jpy
53608,"знания, идеи, технологии, необходимые для прои...",66,N32995,ноу-хау,know-how,know-how,n32995,know-how,1,know-how
50116,,74,N31674,жидкокристаллический дисплей,lcd,lcd,n31674,lcd,1,lcd
76514,,89,N41656,сим-карта,sim-карта,sim-карта,n41656,sim-карта,1,sim-карта


In [113]:
kek = set(lol[lol.main_id_x == 'N24098'].loc[:,['definition','root_name','text','lemma','main_word','name']].values.flatten())

In [114]:
kek

{'eur',
 'валюта',
 'денежная единица в Европейском союзе',
 'евро',
 'евровалюта',
 'европейская валюта',
 'европейский валюта'}

In [112]:
set(kek.flatten())

{'eur',
 'валюта',
 'денежная единица в Европейском союзе',
 'евро',
 'евровалюта',
 'европейская валюта',
 'европейский валюта'}